In [13]:
from lda import guidedlda as glda
import numpy as np
import pandas as pd

## Get Prior Guide Words

In [18]:
import glob

from sklearn.feature_extraction.text import CountVectorizer

filenames = [file for file in glob.glob("../data/topics/lemm_topics/*.txt")]

# TODO: cv

guide_cv = CountVectorizer(input='filename', stop_words='english', max_df=0.9)
guide_X = guide_cv.fit_transform(filenames)

guide_word2id = guide_cv.vocabulary_
guide_vocab = guide_cv.get_feature_names()

with open('../data/topics/lemm_guide_words.txt', encoding='utf-8') as f:
    basic_guide_words = [[t for t in topics.split() if t in guide_vocab] for topics in f.read().splitlines()]

## Read in Data

In [14]:
number = 5933
lemmatized_data = pd.read_csv(f'../data/reviews/lemmatized_reviews/lemmatized_reviews_{number}.csv', index_col=0)
lemmatized_data

,tconst,username,rating,review
0,tt0000574,David-240,10/10,movie be believe to be the world first feature...
1,tt0000574,F Gwynplaine MacIntyre,10/10,this afternoon at the barbican i attend the uk...
2,tt0000574,ackstasis,9/10,movie be undoubtedly one of the cinema most si...
3,tt0000574,Ziggy5446,10/10,movie symbolizes both the birth of the austral...
4,tt0000574,Fella_shibby,8/10,this be the original n the first account of ne...
...,...,...,...,...
11743,tt0018621,JohnHowardReid,8/10,paramount groom superbeautiful actress a a rep...
11744,tt0018621,kidboots,8/10,actress seem to be in every other movie during...
11745,tt0018621,F Gwynplaine MacIntyre,7/10,actress be an attractive and talented actress ...
11746,tt0018638,cliffperriam,8/10,movie exists a a reel silent french mm release...


In [ ]:
# guide_model = glda.GuidedLDA(n_topics=len(basic_guide_words), n_iter=100, random_state=7, refresh=20)

# guide_seed_topics = {}
# for t_id, st in enumerate(basic_guide_words):
#     for word in st:
#         guide_seed_topics[guide_word2id[word]] = t_id
        
# # TODO: cv

# guide_model.fit(guide_X, seed_topics=guide_seed_topics, seed_confidence=0.5)

In [ ]:
# import numpy as np
# import re

# guide_words = []

# # TODO: cv

# guide_n_top_words = 15
# for i, topic_dist in enumerate(guide_model.topic_word_):
#     guide_topic_words = np.array(guide_vocab)[np.argsort(topic_dist)][:-(guide_n_top_words+1):-1]
#     guide_words.append(guide_topic_words)
#     guide_topic = re.findall('\\\\([a-z_]+)', filenames[i])[0]
#     print(f'Topic {guide_topic}:\n{" ".join(guide_topic_words)}\n')

## Generate Corpus Vocabulary and Seed Topics

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
import re 

# TODO: cross validate here for cv params?

cv = CountVectorizer(stop_words='english', max_df=0.6, min_df=30)
X = cv.fit_transform(lemmatized_data.review)
word2id = cv.vocabulary_
vocab = cv.get_feature_names()

guide_words = [[t for t in topics if t in vocab] for topics in basic_guide_words]

seed_topics = {}
for t_id, st in enumerate(guide_words):
    for word in st:
        seed_topics[word2id[word]] = t_id

## Fit Model and Display Top Topic Words

In [21]:
# TODO: cv
model = glda.GuidedLDA(n_topics=len(guide_words), n_iter=100)
model.fit(X, seed_topics=seed_topics, seed_confidence=0.8)

n_top_words = 15
for i, topic_dist in enumerate(model.topic_word_):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic = re.findall('\\\\([a-z_]+)', filenames[i])[0]
    print(f'Topic {topic}:\n{" ".join(topic_words)}\n')

INFO:lda:n_documents: 11748
INFO:lda:vocab_size: 5241
INFO:lda:n_words: 1258159
INFO:lda:n_topics: 10
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -13040031
INFO:lda:<10> log likelihood: -11032737
INFO:lda:<20> log likelihood: -10657324
INFO:lda:<30> log likelihood: -10547771
INFO:lda:<40> log likelihood: -10483495
INFO:lda:<50> log likelihood: -10440221
INFO:lda:<60> log likelihood: -10404575
INFO:lda:<70> log likelihood: -10381834
INFO:lda:<80> log likelihood: -10361764
INFO:lda:<90> log likelihood: -10343784
INFO:lda:<99> log likelihood: -10328408


Topic acting:
character make actress comedy silent scene great act time love like performance just tactor story

Topic attraction:
actress love young woman girl play man director make father star wife old life come

Topic cinematography:
make time director story like silent set horror great use scene watch german character cinema

Topic dialogue:
actress play man make star director girl young come scene woman played end father jack

Topic directing:
director story actress scene make shot german silent herr time character work great use early

Topic editing_effects:
scene make time comedy silent great watch like just train stunt work really way sequence

Topic it_factor:
version actress director silent make best good great scene time story original print star year

Topic plot:
story like make just really good silent watch time end look love say think way

Topic sound_music:
actress silent make version score year story sound director time play print new music screen

Topic theme:
people 